In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
import string
#https://stackoverflow.com/questions/19726663/how-to-save-the-pandas-dataframe-series-data-as-a-figure
import six
import math

In [2]:
tbpath = "../../fits/"
productpath = "../../postfit_derivatives/"

In [ ]:
models = ["fulllinearmodel_fit_table.csv","reducedlinearmodelNegBinom_fit_table.csv",
          "reducedlinearmodelq0_fit_table.csv","reducedlinearmodelq0ctime_fit_table.csv",
         "nonlinearmodelq0ctime_fit_table.csv","nonlinearmodel_fit_table.csv"]

In [3]:
rois = []
for model in models:
    df = pd.read_csv(tbpath + model) #get rois in all tables (some may have failed)
    rois += list(df.roi.unique())
  
rois = list(set(rois))
#sort within US and among other coutries then union back
roi_us = np.sort([i for i in rois if i[:2]=='US'])[::-1]
roi_other = np.sort([i for i in rois if i[:2]!='US'])[::-1]
rois = list(roi_us) + list(roi_other)

In [1]:
#plot side by side 

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    

def plot_table_data(theta,ax):
    for j, roi in enumerate(rois):
        try:
            boxes = [
                {
                'x': i,
                'label' : roi,
                'whislo': df.loc[(df.roi==roi)&(df['quantile']==0.025), col].values[0],    # Bottom whisker position
                'q1'    : df.loc[(df.roi==roi)&(df['quantile']==0.25), col].values[0],    # First quartile (25th percentile)
                'med'   : df.loc[(df.roi==roi)&(df['quantile']==0.5), col].values[0],    # Median         (50th percentile)
                'q3'    : df.loc[(df.roi==roi)&(df['quantile']==0.75), col].values[0],     # Third quartile (75th percentile)
                'whishi': df.loc[(df.roi==roi)&(df['quantile']==0.975), col].values[0],    # Top whisker position
                'fliers': []        # Outliers
                }
            ]
            ax.annotate(roi,-)
            ax.bxp(boxes, positions=[j], showfliers=False, vert=False, patch_artist=True,
                   boxprops=dict(facecolor='none',edgecolor='gray'))
        except:
            print(roi)
            
f,ax = plt.subplots(1,1,figsize=((10,5))

plot_table_data(theta,ax)
plt.subplots_adjust(hspace=0.2,wspace=1)
# plt.savefig(productpath + theta + '_forestplot.png')


SyntaxError: invalid syntax (<ipython-input-1-00775f99d2be>, line 28)